In [3]:
import random
import pandas as pd
import math
import numpy as np
from data_process import get_CIFAR10_data
%matplotlib inline
from save_submission import output_submission_csv
import time

# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 50000]
, TEST_IMAGES = 10000

In [4]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 40000 #40000
VAL_IMAGES = 10000 #10000

In [5]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES)
X_train_CIFAR, y_train_CIFAR = data['X_train'], data['y_train']
X_val_CIFAR, y_val_CIFAR = data['X_val'], data['y_val']
X_test_CIFAR, y_test_CIFAR = data['X_test'], data['y_test']
n_class_CIFAR = len(np.unique(y_test_CIFAR))

Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [6]:
X_train_CIFAR = np.reshape(X_train_CIFAR, (X_train_CIFAR.shape[0], -1))
X_val_CIFAR = np.reshape(X_val_CIFAR, (X_val_CIFAR.shape[0], -1))
X_test_CIFAR = np.reshape(X_test_CIFAR, (X_test_CIFAR.shape[0], -1))

# Get Accuracy
This function computes how well your model performs using accuracy as a metric.


In [7]:
def get_acc(pred, y_test):
    return np.sum(y_test==pred)/len(y_test)*100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.


The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


# Model Perceptron

In [29]:
import numpy as np


class Perceptron:
    def __init__(self, n_class: int, lr: float, epochs: int):

        self.w = np.random.randn(10, 3073)
        self.accuracy_train_accumulated = []
        self.accuracy_val_accumulated = []
        self.lr = lr
        self.epochs = epochs
        self.n_class = n_class

    def train(self, X_train: np.ndarray, y_train: np.ndarray, X_val: np.ndarray, y_val: np.ndarray):

        N, D = X_train.shape
        X_train = np.hstack((np.ones((N,1)), X_train))
        N, D = X_train.shape
        N_val, D_val = X_val.shape
        X_val = np.hstack((np.ones((N_val,1)), X_val))
        N_val, D_val = X_val.shape
        self.w = np.random.randn(self.n_class, D)
        y_pred_train = np.zeros(N)
        y_pred_val = np.zeros(N_val)

        ###### YOUR CODE STARTS HERE ######
        
        
        for ep in range(self.epochs):
            start = time.time()
            for t in range(N):
                y_i = np.argmax(np.dot(self.w, X_train[t]))
                current_class = y_train[t]
                
                if y_i != current_class:
                    self.w[current_class] += self.lr*X_train[t]
                    self.w[y_i] = self.w[y_i] - self.lr*X_train[t]
            
            for t in range(N):
                y_pred_train[t] = np.argmax(np.dot(self.w, X_train[t]))
            
            for t in range(N_val):
                y_pred_val[t] = np.argmax(np.dot(self.w, X_val[t]))  
                
                    
            accuracy_train = (sum(y_train == y_pred_train))/X_train.shape[0]
            self.accuracy_train_accumulated.append(accuracy_train)
            
            accuracy_val = (sum(y_val == y_pred_val))/X_val.shape[0]
            self.accuracy_val_accumulated.append(accuracy_val)
            
            stop = time.time()
            print(f'for Epoch {ep}, Training Accuracy: {accuracy_train:.3f}, Time per Epoch: {(stop-start):.2f} seconds')
        
        ###### YOUR CODE ENDS HERE ######
        return self.accuracy_train_accumulated, self.accuracy_val_accumulated

    def predict(self, X_test: np.ndarray) -> np.ndarray:

        N, D = X_test.shape
        X_test = np.hstack((np.ones((N,1)), X_test))
        N, D = X_test.shape
        y_test = np.zeros(N)
        ###### YOUR CODE STARTS HERE ######
        
        for t in range(N):
            y_test[t] = np.argmax(np.dot(self.w, X_test[t]))
            
        ###### YOUR CODE ENDS HERE ######
        return y_test
    

## Train Perceptron on CIFAR

In [31]:
lr = 0.5
n_epochs = 30
n_class_CIFAR = 10

percept_CIFAR = Perceptron(n_class_CIFAR, lr, n_epochs)
accumulated_train_pred, accumulated_val_pred = percept_CIFAR.train(X_train_CIFAR, y_train_CIFAR, X_val_CIFAR, y_val_CIFAR)

for Epoch 0, Training Accuracy: 0.277, Time per Epoch: 4.69 seconds
for Epoch 1, Training Accuracy: 0.310, Time per Epoch: 2.56 seconds
for Epoch 2, Training Accuracy: 0.298, Time per Epoch: 2.87 seconds
for Epoch 3, Training Accuracy: 0.286, Time per Epoch: 2.43 seconds
for Epoch 4, Training Accuracy: 0.301, Time per Epoch: 2.66 seconds
for Epoch 5, Training Accuracy: 0.313, Time per Epoch: 2.51 seconds
for Epoch 6, Training Accuracy: 0.325, Time per Epoch: 3.06 seconds
for Epoch 7, Training Accuracy: 0.321, Time per Epoch: 2.54 seconds
for Epoch 8, Training Accuracy: 0.309, Time per Epoch: 2.16 seconds
for Epoch 9, Training Accuracy: 0.316, Time per Epoch: 2.62 seconds
for Epoch 10, Training Accuracy: 0.325, Time per Epoch: 7.89 seconds
for Epoch 11, Training Accuracy: 0.299, Time per Epoch: 2.70 seconds
for Epoch 12, Training Accuracy: 0.321, Time per Epoch: 4.09 seconds
for Epoch 13, Training Accuracy: 0.336, Time per Epoch: 4.52 seconds
for Epoch 14, Training Accuracy: 0.325, Time

In [122]:
accumulated_train_pred

[0.27735,
 0.3098,
 0.298,
 0.28625,
 0.3015,
 0.312525,
 0.325075,
 0.321225,
 0.309,
 0.31585,
 0.3247,
 0.2987,
 0.321325,
 0.336375,
 0.324525,
 0.317925,
 0.326475,
 0.33525,
 0.3252,
 0.31465,
 0.326625,
 0.308625,
 0.32055,
 0.333475,
 0.324225,
 0.312975,
 0.32305,
 0.33615,
 0.317125,
 0.328675]

In [123]:
accumulated_val_pred

[0.257,
 0.2853,
 0.2699,
 0.254,
 0.2594,
 0.2773,
 0.2851,
 0.2723,
 0.2722,
 0.2736,
 0.2791,
 0.2571,
 0.2707,
 0.2878,
 0.2818,
 0.2766,
 0.2743,
 0.2892,
 0.2713,
 0.2647,
 0.2813,
 0.2546,
 0.2688,
 0.2728,
 0.2716,
 0.2659,
 0.2733,
 0.2761,
 0.2623,
 0.2684]

In [634]:
pred_percept = percept_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_CIFAR)))

The training accuracy is given by: 30.655000


### Validate Perceptron on CIFAR

In [624]:
pred_percept = percept_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_CIFAR)))

The validation accuracy is given by: 26.280000


### Test Perceptron on CIFAR

In [111]:
pred_percept = percept_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_CIFAR)))

The testing accuracy is given by: 26.850000


In [113]:
output_submission_csv('output/M_Furkan_Oruc_Perceptron_submission_CIFAR.csv', percept_CIFAR.predict(X_test_CIFAR))

# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.




The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [98]:
"""Logistic regression model."""

import numpy as np
import math


class Logistic:
    def __init__(self, lr: float, epochs: int):

        self.w = np.random.randn(10, 3072)
        self.b = np.random.uniform(-0.0001, 0.0001, (1,10))
        self.lr = lr
        self.epochs = epochs
        self.accuracy_train_accumulated = []
        self.accuracy_val_accumulated = []
        #self.threshold = ###### TODO: change this

    def softmax(self, z: np.ndarray) -> np.ndarray:

        ##### YOUR CODE STARTS HERE #####
        soft_max = np.exp(z)/(np.exp(z).sum())
        
        ##### YOUR CODE ENDS HERE #####
        return soft_max

    def train(self, X_train: np.ndarray, y_train: np.ndarray, X_val: np.ndarray, y_val: np.ndarray):

        N, D = X_train.shape
        N_val, D_val = X_val.shape
        #self.w = np.random.randn(10, D)
        o = np.random.randn(1, 10)
        y = np.empty_like(self.w)
        r_train_kronecker = pd.get_dummies(y_train).to_numpy()
        classes_train = np.zeros(N)
        y_pred_val = np.zeros(N_val)

        ##### YOUR CODE STARTS HERE #####
        
        for ep in range(self.epochs):
            start = time.time()
            delta_w = np.zeros_like(self.w)
            delta_b = np.zeros_like(o)
            
            for t in range(N):
                o = (self.w@X_train[t,:]) + self.b
            
                o = o.reshape(1,10)
                o -= np.max(o)
                y = self.softmax(o)
            
                delta_w = delta_w + ((r_train_kronecker[t,:].reshape(1,10)-y)).T@X_train[t,:].reshape(1,D)
                delta_b[0,:] = delta_b[0,:] + ((r_train_kronecker[t,:].reshape(1,10)-y))
                
                classes_train[t] = np.argmax(y, axis = 1)
                

                self.w = self.w + self.lr*delta_w
                self.b = self.b + self.lr*delta_b
        
            classes = np.argmax(y, axis = 1)
                
            accuracy_train = (sum(y_train == classes_train))/X_train.shape[0]
            self.accuracy_train_accumulated.append(accuracy_train)
            
            for t in range(N_val):
                o = (self.w@X_val[t,:]) + self.b
                o = o.reshape(1,10)
                y = self.softmax(o)
                y_pred_val[t] = np.argmax(y,axis=1)

            
            accuracy_val = (sum(y_val == y_pred_val))/X_val.shape[0]
            self.accuracy_val_accumulated.append(accuracy_val)

            stop = time.time()
            print(f'for Epoch {ep}, Training Accuracy: {accuracy_train:.3f}, Time per Epoch: {(stop-start):.2f} seconds')

        ##### YOUR CODE ENDS HERE #####
        return self.accuracy_train_accumulated, self.accuracy_val_accumulated

    def predict(self, X_test: np.ndarray) -> np.ndarray:

        N, D = X_test.shape
        y_test = np.zeros((1,N))
        classes_test = np.zeros((1,N))
        ##### YOUR CODE STARTS HERE #####
        
        for t in range(N):
            o = (self.w@X_test[t,:]) + self.b
            
            o = o.reshape(1,10)
            o -= np.max(o)
            y = self.softmax(o)
            classes_test[0,t] = np.argmax(y,axis=1)
            
        #accuracy = get_acc(classes_test, y_test)
        
        ##### YOUR CODE ENDS HERE #####
        return classes_test

### Training Logistic Classifer

In [116]:
learning_rate = 0.75
n_epochs = 20
#0.01
#10

lr = Logistic(learning_rate, n_epochs)
accumulated_train_pred_lr, accumulated_val_pred_lr = lr.train(X_train_CIFAR, y_train_CIFAR, X_val_CIFAR, y_val_CIFAR)

/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp
/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


for Epoch 0, Training Accuracy: 0.259, Time per Epoch: 15.71 seconds
for Epoch 1, Training Accuracy: 0.274, Time per Epoch: 12.38 seconds
for Epoch 2, Training Accuracy: 0.275, Time per Epoch: 17.00 seconds
for Epoch 3, Training Accuracy: 0.285, Time per Epoch: 26.74 seconds
for Epoch 4, Training Accuracy: 0.288, Time per Epoch: 20.78 seconds
for Epoch 5, Training Accuracy: 0.286, Time per Epoch: 16.12 seconds
for Epoch 6, Training Accuracy: 0.283, Time per Epoch: 15.93 seconds
for Epoch 7, Training Accuracy: 0.287, Time per Epoch: 17.00 seconds
for Epoch 8, Training Accuracy: 0.296, Time per Epoch: 21.66 seconds
for Epoch 9, Training Accuracy: 0.296, Time per Epoch: 13.45 seconds
for Epoch 10, Training Accuracy: 0.290, Time per Epoch: 13.74 seconds
for Epoch 11, Training Accuracy: 0.302, Time per Epoch: 13.34 seconds
for Epoch 12, Training Accuracy: 0.296, Time per Epoch: 13.85 seconds
for Epoch 13, Training Accuracy: 0.292, Time per Epoch: 13.17 seconds
for Epoch 14, Training Accurac

In [124]:
learning_rate = 2
n_epochs = 5
#0.01
#10

lr = Logistic(learning_rate, n_epochs)
accumulated_train_pred_lr, accumulated_val_pred_lr = lr.train(X_train_CIFAR, y_train_CIFAR, X_val_CIFAR, y_val_CIFAR)

/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp
/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


for Epoch 0, Training Accuracy: 0.259, Time per Epoch: 13.91 seconds
for Epoch 1, Training Accuracy: 0.274, Time per Epoch: 13.56 seconds
for Epoch 2, Training Accuracy: 0.275, Time per Epoch: 14.76 seconds
for Epoch 3, Training Accuracy: 0.285, Time per Epoch: 13.52 seconds
for Epoch 4, Training Accuracy: 0.288, Time per Epoch: 12.97 seconds


In [117]:
accumulated_train_pred_lr

[0.259475,
 0.273575,
 0.27475,
 0.284525,
 0.287525,
 0.285675,
 0.2829,
 0.2866,
 0.295975,
 0.295825,
 0.290275,
 0.3022,
 0.296175,
 0.2924,
 0.29235,
 0.293,
 0.28825,
 0.2956,
 0.2926,
 0.30065]

In [118]:
accumulated_val_pred_lr

[0.1679,
 0.1544,
 0.1805,
 0.1599,
 0.1604,
 0.1585,
 0.1332,
 0.1545,
 0.1628,
 0.1491,
 0.1592,
 0.1534,
 0.1608,
 0.1711,
 0.1655,
 0.1619,
 0.169,
 0.1609,
 0.1618,
 0.1437]

In [125]:
pred_lr = lr.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_CIFAR)))

/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp
/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


The training accuracy is given by: 16.197500


### Validate Logistic Classifer

In [126]:
pred_lr = lr.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_CIFAR)))

/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp
/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


The validation accuracy is given by: 16.040000


### Test Logistic Classifier

In [127]:
pred_lr = lr.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_CIFAR)))
output_submission_csv('output/M_Furkan_Oruc_Logistic_submission_CIFAR.csv', lr.predict(X_test_CIFAR))

/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp
/Users/furkanoruc/opt/anaconda3/envs/introDL/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


The testing accuracy is given by: 15.680000


### Problem #8

In [62]:
y = np.arange(start=-20, stop=20.1, step=0.1).reshape((401, 1))

In [63]:
j = np.arange(start=1, stop=51, step=1.0).reshape((1, 50))
n

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50.]])

In [67]:
result = y**j
print(result)
print(result.shape)

[[-2.00000000e+01  4.00000000e+02 -8.00000000e+03 ...  2.81474977e+62
  -5.62949953e+63  1.12589991e+65]
 [-1.99000000e+01  3.96010000e+02 -7.88059900e+03 ...  2.21282805e+62
  -4.40352782e+63  8.76302035e+64]
 [-1.98000000e+01  3.92040000e+02 -7.76239200e+03 ...  1.73751728e+62
  -3.44028422e+63  6.81176275e+64]
 ...
 [ 1.98000000e+01  3.92040000e+02  7.76239200e+03 ...  1.73751728e+62
   3.44028422e+63  6.81176275e+64]
 [ 1.99000000e+01  3.96010000e+02  7.88059900e+03 ...  2.21282805e+62
   4.40352782e+63  8.76302035e+64]
 [ 2.00000000e+01  4.00000000e+02  8.00000000e+03 ...  2.81474977e+62
   5.62949953e+63  1.12589991e+65]]
(401, 50)
